In [8]:
# Baseline은 test score의 기준값(기준선)을 잡기 위한 가장 기본 프로세스

In [129]:
import numpy as np
import pandas as pd
import os

In [128]:
from catboost.datasets import amazon

train, test = amazon()

In [130]:
print("Train shape : {}. Test shape : {}".format(train.shape, test.shape))

Train shape : (32769, 10). Test shape : (58921, 10)


In [131]:
train.head(5)

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [25]:
test.head(5)

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [18]:
train.apply(lambda x: len(x.unique()))

ACTION                 2
RESOURCE            7518
MGR_ID              4243
ROLE_ROLLUP_1        128
ROLE_ROLLUP_2        177
ROLE_DEPTNAME        449
ROLE_TITLE           343
ROLE_FAMILY_DESC    2358
ROLE_FAMILY           67
ROLE_CODE            343
dtype: int64

In [ ]:
# 모두 범주형 고유값 개수가 너무 많으면 0,1로 매칭하기가 어려울 것 (개수가 많을수록 까다로운 모델)
# 개수가 같다 = 1:1 매핑이지 않을까? -> 확인하여서 날린다

In [34]:
col1 = "ROLE_TITLE"
col2 = "ROLE_CODE"
col1, col2

('ROLE_TITLE', 'ROLE_CODE')

In [49]:
target = 'ACTION'
col4train = [x for x in train.columns if x != target]

In [50]:
pair = len(train.groupby([col1, col2]).size())
single = len(train.groupby(col1).size())

In [51]:
pair, single

(343, 343)

In [52]:
col4train = [x for x in col4train if x != 'ROLE_TITLE']

In [53]:
col4train

['RESOURCE',
 'MGR_ID',
 'ROLE_ROLLUP_1',
 'ROLE_ROLLUP_2',
 'ROLE_DEPTNAME',
 'ROLE_FAMILY_DESC',
 'ROLE_FAMILY',
 'ROLE_CODE']

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [57]:
ohe = OneHotEncoder(sparse = True, dtype=np.float32, handle_unknown = 'ignore')

In [81]:
help(OneHotEncoder)

Help on class OneHotEncoder in module sklearn.preprocessing._encoders:

class OneHotEncoder(_BaseEncoder)
 |  OneHotEncoder(*, categories='auto', drop=None, sparse=True, dtype=<class 'numpy.float64'>, handle_unknown='error')
 |  
 |  Encode categorical features as a one-hot numeric array.
 |  
 |  The input to this transformer should be an array-like of integers or
 |  strings, denoting the values taken on by categorical (discrete) features.
 |  The features are encoded using a one-hot (aka 'one-of-K' or 'dummy')
 |  encoding scheme. This creates a binary column for each category and
 |  returns a sparse matrix or dense array (depending on the ``sparse``
 |  parameter)
 |  
 |  By default, the encoder derives the categories based on the unique values
 |  in each feature. Alternatively, you can also specify the `categories`
 |  manually.
 |  
 |  This encoding is needed for feeding categorical data to many scikit-learn
 |  estimators, notably linear models and SVMs with the standard ker

In [63]:
X = ohe.fit_transform(train[col4train])
y = train[target].values

In [2]:
# RMSE ( 예측 값을 보정해주는 방식 = 예측값이 잘 못 나오는 것을 보정해줌! / 예측에 대해 penalty를 적게 주겠다!!! )
# MAE ( 있는 그대로 내뱉겠다 ) ( penalty를 크게 주겠다 )
# L2를 사용한 이유??? -> 

In [68]:
from sklearn.model_selection import cross_validate

In [4]:
model = LogisticRegression(penalty='l2'     # l2 = Ridge(제곱) / l1 = Lasso(절대값)
                           ,C=1.0
                           ,fit_intercept='True'
                           ,random_state=432
                           ,solver='liblinear'
                           ,max_iter=1000
)
# overfitting 여부 확인하기 위해 cross_validate()
stats = cross_validate(model, X, y, groups=None, scoring='roc_auc'  # AUC(Area under the curve):임의의 curve 아래 면적을 계산 (ROC curve의 AUD를 계산)
                       ,cv=5, n_jobs=2, return_train_score= True)   # cv = K-ford의 k값 / n_jobs = multiprocessor 개수(-1는 쓰지말것 = 모두 쓰는 것)
# yes or no 만 단순히 맞추는것이 아니라 TP, FN, TN, FP 까지 모두 비교한 비율을 구하는 것이 ROC curve(Binary 분류에서 많이 쓰임!!!)
stats = pd.DataFrame(stats)
stats.describe().transpose()

NameError: name 'LogisticRegression' is not defined

In [5]:
# FInd out LB score
X = ohe.fit_transform(train[col4train])
y = train[target].values                     # y(반응변수)는 train set의 target 칼럼의 값
X_te = ohe.transform(test[col4train])        # test set의 X는 training에 계산된 동일한 평균, 표준편차로 정규화(transform)함
# Nomalization : 
# fit : train set의 범위에서 평균, 표준편차를 구하는 것 (어떤 범위에서 fit할거냐!? -> train set)
# transform : val, test 범위에서 fit했던 내용으로 nomalization하는 것
# fit_transform : 같이 하겠다 (train set에서만 해야하는 것)

NameError: name 'ohe' is not defined

In [119]:
# 모델 학습 및 예측
model.fit(X,y)
pred = model.predict_proba(X_te)      # y의 각 클래스별 확률(ROC_AUC를 구하기 위해 필요함)을 구함
predictions = pred[:,1]               # 두번째(2?)클래스의 확률값만 뽑음
pred                                  # 1, 2번째 클래스 확률 값의 합은 1
predictions                           # 두 번째 클래스 확률 값만 배열로 뽑는다

array([0.84642342, 0.98278272, 0.98869776, ..., 0.98860966, 0.89406542,
       0.99439442])

In [120]:
# Test dataset에 대한 예측값을 predictions에 넣어 submission.csv로 추출
submit = pd.DataFrame()
submit['ID'] = test['id']
submit['ACTION'] = predictions                
submit.to_csv('submission.csv',index=False)